<a href="https://colab.research.google.com/github/Maansi-1590/Aerial-Object-Project/blob/main/Aerial_Object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ▣ Check GPU
!nvidia-smi

# ▣ Install Required Libraries
!pip install tensorflow keras matplotlib seaborn scikit-learn
!pip install ultralytics
!pip install streamlit

/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.9 MB/s eta 0:00:00
